In [1]:
%run helper/setup_notebook.ipynb

Successfully connected to sql_lab database.


#### ***Pivot tables*** enable the presentation of row values as columns, facilitating insightful observations. Unfortunately, MySQL does not provide a built-in function for creating pivot tables. Nevertheless, there exist multiple approaches to creating pivot tables in MySQL.

In [2]:
%%sql

SELECT * 
FROM exams;

student,exam,score
Bao,Math,90
Bao,English,78
Bao,History,88
Bao,Art,60
Joe,Math,69
Joe,English,87
Joe,History,98
Joe,Art,79
Jane,Math,99
Jane,English,77


### *Desired output*

```sql
+---------+------+---------+---------+------+
| student | Math | English | History | Art  |
+---------+------+---------+---------+------+
| Bao     |   90 |      78 |      88 |   60 |
| Jane    |   99 |      77 |      83 |  100 |
| Joe     |   69 |      87 |      98 |   79 |
+---------+------+---------+---------+------+
```

## Using *CASE* Statement

In [3]:
%%sql 

SELECT
    student,
    CASE WHEN exam='Math' THEN score ELSE NULL END AS Math,
    CASE WHEN exam='English' THEN score ELSE NULL END AS English,
    CASE WHEN exam='History' THEN score ELSE NULL END AS History,
    CASE WHEN exam='Art' THEN score ELSE NULL END AS Art 
FROM exams;

student,Math,English,History,Art
Bao,90,None,None,None
Bao,None,78,None,None
Bao,None,None,88,None
Bao,None,None,None,60
Joe,69,None,None,None
Joe,None,87,None,None
Joe,None,None,98,None
Joe,None,None,None,79
Jane,99,None,None,None
Jane,None,77,None,None


In [4]:
%%sql 

-- since we are dealing with integer values 
-- we can group by student name to use SUM function

SELECT
    student,
    SUM(CASE WHEN exam='Math' THEN score ELSE NULL END) AS Math,
    SUM(CASE WHEN exam='English' THEN score ELSE NULL END) AS English,
    SUM(CASE WHEN exam='History' THEN score ELSE NULL END) AS History,
    SUM(CASE WHEN exam='Art' THEN score ELSE NULL END) AS Art 
FROM exams
GROUP BY student;

student,Math,English,History,Art
Bao,90,78,88,60
Joe,69,87,98,79
Jane,99,77,83,100


In [5]:
%%sql 

-- pivot tables enable to derive valuable insight
-- assume that we want to find out the average score of the students on
-- non-Math related exams 

SELECT 
    student,
    SUM(CASE WHEN exam <> 'Math' THEN score ELSE NULL END)/3 AS Non_Math_Average,
    SUM(CASE WHEN exam = 'Math' THEN score ELSE NULL END) AS Math 
FROM exams
GROUP BY student;

student,Non_Math_Average,Math
Bao,75.3333,90
Joe,88.0000,69
Jane,86.6667,99


## Using *IF* Statement

In [6]:
%%sql 

SELECT
    student,
    SUM(IF(exam='Math', score, NULL)) AS Math,
    SUM(IF(exam='English', score, NULL)) AS English,
    SUM(IF(exam='History', score, NULL)) AS History,
    SUM(IF(exam='Art', score, NULL)) AS Art 
FROM exams 
GROUP BY student;


student,Math,English,History,Art
Bao,90,78,88,60
Joe,69,87,98,79
Jane,99,77,83,100


## Using *Stored Procedure*

- #### These two methods work great if we know the column names - `Math`, `English`, etc.
- #### *GROUP_CONCAT* function helps us dynamically generate the columns of a PIVOT table. 

In [7]:
%%sql

DROP PROCEDURE IF EXISTS create_pivot_table;

CREATE PROCEDURE create_pivot_table()
BEGIN
    DECLARE sql_query VARCHAR(4000);
    DECLARE exam_list VARCHAR(4000);
  
    -- Get the list of unique exams
    SELECT 
        GROUP_CONCAT(DISTINCT CONCAT('MAX(CASE WHEN exam = ''', exam, ''' THEN score END) AS ', exam)) 
    INTO exam_list 
    FROM exams;
  
    -- Prepare the dynamic SQL query
    SET @sql_query = CONCAT('SELECT student, ', exam_list, ' FROM exams GROUP BY student;');
  
    -- Execute the dynamic SQL query
    PREPARE stmt FROM @sql_query;
    EXECUTE stmt;
    DEALLOCATE PREPARE stmt;
END;


[]

In [8]:
%%sql
-- unable to call a procedure
-- CALL create_pivot_table();

[]